In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import xml.etree.ElementTree as ET

In [ ]:
sheep_ages = pd.read_csv('sheep_data/sheep_age.csv')
sheep_edges = pd.read_csv('sheep_data/sheep_edges.csv')


In [ ]:
sheep_network

In [ ]:
def calc_degrees(gph):
    degrees = dict(gph.degree)
    return degrees

def calc_degree_colors(gph):
    degrees = calc_degrees(gph)
    # colors for degree value
    min_degree = min(degrees.values())
    max_degree = max(degrees.values())
    colors = {node: (degrees[node] - min_degree)/ (max_degree - min_degree) for node in gph.nodes()}
    return colors


def create_dash_app(gph):
    node_degrees = calc_degrees(gph)
    deg_min, deg_max = min(node_degrees.values()), max(node_degrees.values())
    edge_weights = nx.get_edge_attributes(gph, 'weight')
    positions = nx.layout.shell_layout(gph) # spring_layout, random_layout, shell_layout, circular_layout
    app = dash.Dash(__name__)
    
    fig = go.Figure()
    for edge in sheep_network.edges():
        n1pos = positions[edge[0]]
        n2pos = positions[edge[1]]
        fig.add_trace(go.Scatter(
            x=[n1pos[0], n2pos[0]],
            y=[n1pos[1], n2pos[1]],
            mode='lines',
            line=dict(width=2, color='black')
        ))
    
    node_degs = []
    node_sizes = []
    pos_x = []
    pos_y = []
    for node in gph.nodes():
        node_degs.append(node_degrees[node])
        scaled_degree = (node_degrees[node] - deg_min) / (deg_max - deg_min)
        node_sizes.append(5 + 20 * scaled_degree) # scale size of nodes to range [5, 25]
        pos = positions[node]
        pos_x.append(pos[0])
        pos_y.append(pos[1])
    node_df = {
        'Node': gph.nodes(),
        'Degree': node_degs,
        'Size': node_sizes,
        'x': pos_x,
        'y': pos_y,
        
    }
    
    px_nodes = px.scatter(
        node_df,
        x='x',
        y='y',
        size='Size',
        color='Degree',
        color_continuous_scale ='thermal',
        hover_name='Node'
        hover_data={'Degree': True, 'Size': False, 'x': False, 'y': False}
        #hoverinfo='text',
        #text=f'Node {node}, Degree {gph.degree[node]}'
    )
    fig.add_trace(px_nodes.data[0])

    fig.update_layout(
        showlegend=False,
        hovermode='closest'
    )
    
    app.layout = dbc.Container([
        html.H1("Sheep Network Node Links and Degrees", className="mt-4 mb-4"),
        dcc.Graph(figure=fig)
    ])
    return app

if __name__ == '__main__':
    sheep_network = nx.read_graphml('sheep_data/sheep_ml.graphml')
    app = create_dash_app(sheep_network)
    app.run_server(debug=True)

sheep_network = nx.read_graphml('sheep_data/sheep_ml.graphml')
app = create_dash_app(sheep_network)
app.run_server(debug=True)

In [ ]:
app